## Analysis correlating evt and raw

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import awkward as ak
import os
import pint
import glob
import re
from functools import partial

# I use the SiPM QC Analysis kernel (from .venv/bin)

u = pint.get_application_registry()

from lgdo import lh5
from lgdo.lh5 import LH5Store, LH5Iterator, read, read_as, ls, show
from dspeed.vis.waveform_browser import WaveformBrowser
from legendmeta import LegendMetadata

from latools.utils import get_key_for_rawid, get_detector_system_for_channelname, get_filtered_keys_in_detectorsystem
from latools import core
from latools.browse import BrowseTask, BrowseAnydetTask
from latools.histogram import HistogramTask, Histogram2DTask, CategoricalHistogramTask, CategoricalHistogram2DTask
from latools.counter import CountTask

plt.rcParams["figure.figsize"] = (14, 4)
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = 14

proj_dir = "/mnt/atlas02/projects/legend/sipm_qc"
lmeta = LegendMetadata(os.path.join(proj_dir, "metadata/legend-metadata-schwarz"))
chmap = lmeta.channelmap("20250624T140504Z")

In [ ]:
dsp_dir = os.path.join(proj_dir, "manual_dsp/generated/p14r006dsp")
dsp_files = glob.glob(dsp_dir+"/l200-*-tier_dsp.lh5")
dsp_files.sort()

raw_dir = os.path.join(proj_dir, "data/tier/raw/lac/p14/r006")

def gimme_raw_filename_from_dsp(dspfilename: str):
    return dspfilename.replace(dsp_dir, raw_dir).replace("tier_dsp", "tier_raw")

In [ ]:
main_loop = partial(core.main_loop, tier_filename_dict=
                    [{"raw": gimme_raw_filename_from_dsp(dsp), "dsp": dsp} for dsp in dsp_files])

In [ ]:
# These 49 rawids I see in the event tier of p14 r006 lac. Some channels existing in raw & dsp (e.g. S054) don't show up here...
rawids_used_in_evt = [1052803, 1052804, 1054401, 1054402, 1054403, 1054404, 1054405, 1056000, 1056001, 1056003, 1056004, 1056005, 1057601, 1057602, 1057603, 1057604, 1057605, 1059200, 1059201, 1059202, 1059204, 1059205, 1060801, 1060803, 1060804, 1060805, 1062400, 1062401, 1062402, 1062403, 1062404, 1062405, 1064000, 1064001, 1064002, 1064003, 1064004, 1065600, 1065601, 1065602, 1065603, 1065604, 1065605, 1067200, 1067201, 1067202, 1067203, 1067204, 1067205]
usable_sipm_keys = get_filtered_keys_in_detectorsystem(chmap, "spms", rawids_used_in_evt)

In [ ]:
# set up chain's input data
inputArraysDef = []

def generate_SiPM_hit_defs():
    ret = []
    for sipm in ["S055"]: # usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):
        ret += [(f"{sipm}_wf_fwhm", f"{sipm}/dsp/wf_fwhm"), (f"{sipm}_wf_mode", f"{sipm}/dsp/wf_mode"), (f"{sipm}_wf_min_small", f"{sipm}/dsp/wf_min_small")]
        ret += [(f"{sipm}_curr_fwhm", f"{sipm}/dsp/curr_fwhm"), (f"{sipm}_curr_mode", f"{sipm}/dsp/curr_mode"), (f"{sipm}_curr_min_small", f"{sipm}/dsp/curr_min_small")]
    return ret
inputArraysDef += generate_SiPM_hit_defs()

In [ ]:
# set up processing functions
genArrayDef = []

def generate_SiPM_hit_gen_defs():
    ret = []
    for sipm in ["S055"]: # usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):
        ret += [
            ([f"{sipm}_wf_min_small", f"{sipm}_wf_mode"], f"{sipm}_min_rel", lambda x: x[0] - x[1]),
            ([f"{sipm}_min_rel", f"{sipm}_wf_fwhm"], f"{sipm}_any_noise_class", lambda x: (-x[0])/(6*x[1])),
            ([f"{sipm}_curr_min_small", f"{sipm}_curr_mode"], f"{sipm}_curr_min_rel", lambda x: x[0] - x[1]),
            ([f"{sipm}_min_rel", f"{sipm}_curr_fwhm"], f"{sipm}_curr_any_noise_class", lambda x: (-x[0])/(6*x[1])),
        ]
    return ret
genArrayDef += generate_SiPM_hit_gen_defs()

In [ ]:
# Stuff which generates understandable output
browseAnydet = partial(BrowseAnydetTask, channelmap=chmap) # , blacklist=["S058"])

# hint: x[y] removed masked-out entries, removing rows
# ak.mask(x,y) replaces masked-out entries with none, keeping the shape intact

outDef = [
    #(["S055_any_noise_class"], HistogramTask(-5,5, nbins=100, min_entries_required=20000000, logy=True)),
    #(["spms_isphys", "spms_rawid"], BrowseTask(lambda x: ak.any((~x[0]) & (x[1]==chmap["S055"].daq.rawid), axis=-1), "S055", max_entries=10)),
    #(["spms_isphys", "spms_rawid"], BrowseTask(lambda x: ak.any((x[0]) & (x[1]==chmap["S055"].daq.rawid), axis=-1), "S055", max_entries=10)),
    #(["S055_any_noise_class"], BrowseTask(lambda x: x[0]>1, "S055", max_entries=20)),
    #(["S055_any_noise_class", "spms_isphys", "spms_rawid"], Histogram2DTask(-1, 3, 50, -1, 2, 5, 
    #                                                                        lambda x: x[0], lambda x: ak.any((~x[1]) & (x[2]==chmap["S055"].daq.rawid), -1), logz=True)),
    #(["spms_isphys", "spms_rawid", "coincident_puls"], CategoricalHistogramTask(lambda x: ak.flatten(x[1][~x[0]][~x[2]], -1), keymap_fcn=partial(get_key_for_rawid, chmap), min_entries_required=10000)),
    #(["spms_isphys", "spms_rawid", "ge_energy", "ge_rawid", "coincident_puls"], 
    # CategoricalHistogram2DTask(lambda x: x[1][~x[0]], lambda x: ak.mask(x[3][x[2] > 500], ~x[4]),
    #                            mode="cartesian", keymap_fcn=partial(get_key_for_rawid, chmap), min_entries_required=10000)),
    (["S055_curr_min_rel"], BrowseTask(lambda x: x[0] < -3, "S055", max_entries=10)),
    (["S055_curr_min_rel"], BrowseTask(lambda x: x[0] > 3, "S055", max_entries=10)),
    (["S055_curr_fwhm"], BrowseTask(lambda x: x[0] > 8, "S055", max_entries=10)),
    (["S055_curr_min_small"], BrowseTask(lambda x: x[0] > 2, "S055", max_entries=10)),
    #(["S073_min_rel"], BrowseTask(lambda x: x[0] < -40, "S073", max_entries=10)),
    #(["S055_min_rel"], BrowseTask(lambda x: x[0] < -40, "S055", max_entries=10)),
    #(["S071_min_rel"], BrowseTask(lambda x: x[0] < -40, "S071", max_entries=10)),
    #(["S029_min_rel"], BrowseTask(lambda x: x[0] < -40, "S029", max_entries=10)),
    #(["S049_min_rel"], BrowseTask(lambda x: x[0] < -40, "S049", max_entries=10)),
    #(["S051_min_rel"], BrowseTask(lambda x: x[0] < -40, "S051", max_entries=10)),
    #(["S007_min_rel"], BrowseTask(lambda x: x[0] < -40, "S007", max_entries=10)),
]

#for spm in ["S055", "S002", "S094", "S095", "S065", "S052"]:
#    addBrowseForNonphysSiPM(outDef, spm)

In [ ]:
def investigate_sipm(outDef, spm: str, *, wf_min_small_min = 3720, min_rel_min = -20) -> None:
    outDef += [
    #([f"{spm}_any_noise_class"], HistogramTask(-5,5, nbins=100, min_entries_required=20000000, logy=True)),
    #([f"{spm}_curr_any_noise_class"], HistogramTask(-5,5, nbins=100, min_entries_required=20000000, logy=True)),
    ([f"{spm}_curr_fwhm"], HistogramTask(0,20, nbins=200, min_entries_required=20000000, logy=True)),
    ([f"{spm}_curr_mode"], HistogramTask(-2,2, nbins=100, min_entries_required=20000000, logy=True)),
    ([f"{spm}_curr_min_small"], HistogramTask(-5,5, nbins=100, min_entries_required=20000000, logy=True)),
    ([f"{spm}_curr_min_rel"], HistogramTask(-5,5, nbins=100, min_entries_required=20000000, logy=True)),
    ([f"{spm}_curr_fwhm", f"{spm}_curr_min_rel"], Histogram2DTask(0,2,200, -5,5,100, min_entries_required=20000000, logz=True)),
    #([f"{spm}_curr_min_rel"], HistogramTask(-5,5, nbins=100, min_entries_required=20000000, logy=True)),
    #([f"{spm}_any_noise_class"], CountTask(lambda x: x[0] > 1, name=f"{spm}_any_noise_class>1")),
    #([f"{spm}_any_noise_class", f"{spm}_wf_mode"], Histogram2DTask(-1, 2.5, 50, 3730, 3750, 50, lambda x: x[0], lambda x: x[1], logz=True)),
    #([f"{spm}_any_noise_class", f"{spm}_wf_min_small"], Histogram2DTask(-1, 2.5, 50, wf_min_small_min, 3750, 50, lambda x: x[0], lambda x: x[1], logz=True)),
    #([f"{spm}_any_noise_class", f"{spm}_min_rel"], Histogram2DTask(-1, 2.5, 50, min_rel_min, 20, 50, lambda x: x[0], lambda x: x[1], logz=True)),
    #([f"{spm}_any_noise_class", f"{spm}_wf_fwhm"], Histogram2DTask(-1, 2.5, 50, 0, 5, 200, lambda x: x[0], lambda x: x[1], logz=True)), #fwhm max was 15
    ]
#investigate_sipm(outDef, "S047", wf_min_small_min=3700, min_rel_min=-42)
#investigate_sipm(outDef, "S015", wf_min_small_min=3700, min_rel_min=-42)
#investigate_sipm(outDef, "S003", wf_min_small_min=3700, min_rel_min=-42)
#investigate_sipm(outDef, "S095", wf_min_small_min=3700, min_rel_min=-42)
#investigate_sipm(outDef, "S065", wf_min_small_min=3700, min_rel_min=-42)
investigate_sipm(outDef, "S055", wf_min_small_min=3700, min_rel_min=-42)


In [ ]:
if False:
    outDef = [] # start from scratch
    fig, ax = plt.subplots()
    for sipm in usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):
        outDef.append( ([f"{sipm}_any_noise_class"], HistogramTask(-5,5, nbins=100, min_entries_required=20000000, logy=True, ax=ax, label=sipm)) )
        outDef.append( ([f"{sipm}_any_noise_class"], CountTask(lambda x: x[0] > 1, name=f"{sipm}_any_noise_class>1")) )
if False:
    fig, ax = plt.subplots()
    for sipm in usable_sipm_keys: # get_keys_in_detectorsystem(chmap, "spms"):  ["S029", "S094", "S047", "S048"]
        outDef.append( ([f"{sipm}_min_rel"], HistogramTask(-250,250, nbins=500, min_entries_required=20000000, logy=True, ax=ax, label=sipm)) )
        outDef.append( ([f"{sipm}_min_rel"], CountTask(lambda x: x[0] < -40, name=f"{sipm}_min_rel<-40")) )

In [ ]:
main_loop(inputArraysDef, genArrayDef, outDef, crop=True)

In [ ]:
# PLAYGROUND ------------

In [ ]:
one = ak.Array([1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 6.6, 6.66])
two = ak.Array([10,  11,  12,  13,  14,  15,  15,  15])

lorem = ak.zip([one, two])
#np.unique(lorem)
lorem
np.unique(lorem, return_counts=True)

In [ ]:
one = ak.Array([[1,2], [], [3], [4,5], [6]])
two = ak.Array([[99], [98, 97], [96, 95], [90, 80], []])
ak.flatten(ak.cartesian([one, two], axis=-1), axis=-1)

In [ ]:
ys = 2
xs = 3
draw_array = np.ndarray((xs, ys))
n = 0
for x in range(len(draw_array)):
    for y in range(len(draw_array[x])):
        draw_array[x][y] = n
        n += 1
draw_array

In [ ]:
fig, ax = plt.subplots()
xedges = [0, 1, 2]
yedges = [0, 1]
ax.pcolor(xedges, yedges, draw_array.T)
ax.set_xticks([0, 1, 2], ["lorem", "ipsum", "dolor"])
ax.set_yticks([0, 1], ["TC-call", "Group meeting"])

In [ ]:
range(4)

In [ ]:
one = ak.Array([[[1,2], [3,4]], None, [[3, 7], [4,5]], [[6,7], [7,5]]])
msk = ak.Array([[True, False], None, [True, False], [True, False]])
ak.sum(one, axis=-1)[msk][:,0] # select the 1st from each inner list, keeping outer None around (other than flatten or ravel)

In [ ]:
ak.sum(one, axis=-1)